In [14]:
import numpy as np
import pytest
import numpy.typing as npt

In [15]:
class NormalFormGameCalculator:

    # non zero sum game constructor
    def __init__(self,
                 row_player_utility_matrix: npt.NDArray[np.float64],
                 column_player_utility_matrix: npt.NDArray[np.float64]) -> None:

        # if col player utility is not provided, we consider this a zero sum game
        if(column_player_utility_matrix is None):
            column_player_utility_matrix = -row_player_utility_matrix

        self.row_player_utility_matrix = row_player_utility_matrix
        self.column_player_utility_matrix = column_player_utility_matrix


    # calculation section

    def calculate_utilities(self,
                            row_player_strategy: npt.NDArray[np.float64],
                            column_player_strategy: npt.NDArray[np.float64]) -> [np.float64, np.float64]:


        action_probabilities = row_player_strategy @ column_player_strategy
        assert action_probabilities.sum() == pytest.approx(1)

        row_player_utility = action_probabilities * self.row_player_utility_matrix
        column_player_utility = action_probabilities * self.column_player_utility_matrix

        return row_player_utility.sum(), column_player_utility.sum()

    def get_best_response_strategy_against_row_player(self,
                                                  row_player_strategy: npt.NDArray[np.float64],
                                                  column_utility_matrix: npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:

        array_of_zeros = np.zeros(column_utility_matrix.shape[0])
        utilities = column_utility_matrix @ row_player_strategy
        index = np.argmax(utilities)
        array_of_zeros[index] = 1
        best_response = np.reshape(a=array_of_zeros, newshape=(1,3))
        return (best_response)
    
    def get_best_response_strategy_against_column_player(self,
                                                  column_player_strategy: npt.NDArray[np.float64],
                                                  row_utility_matrix: npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
        
        array_of_zeros = np.zeros(row_utility_matrix.shape[1])
        utilities = column_player_strategy @ row_utility_matrix
        index = np.argmax(utilities)
        array_of_zeros[index] = 1
        best_response = np.reshape(a=array_of_zeros, newshape=(3,1))
        return (best_response)
    




In [16]:
rock_paper_scissors__utility_matrix = np.array([[0, 1, -1], [-1, 0, 1], [1, -1, 0]])

column_strategy = np.array([[0.3, 0.2, 0.5]])

row_strategy = np.array([[0.1,
                          0.2,
                          0.7]]).transpose()


normal_game_calculator = NormalFormGameCalculator(rock_paper_scissors__utility_matrix, None)

row_util, _ = normal_game_calculator.calculate_utilities(row_strategy, column_strategy)

assert row_util == pytest.approx(0.08)

row_util

0.08000000000000002

In [17]:


best_col = normal_game_calculator.get_best_response_strategy_against_row_player(row_strategy, rock_paper_scissors__utility_matrix)
best_row = normal_game_calculator.get_best_response_strategy_against_column_player(column_strategy, -rock_paper_scissors__utility_matrix)

# values when facing best responding opponent
rowvalue2, _ = normal_game_calculator.calculate_utilities(row_strategy, best_col)
_, colvalue1 = normal_game_calculator.calculate_utilities(best_row, column_strategy)

assert rowvalue2 == pytest.approx(-0.6)
assert colvalue1 == pytest.approx(-0.2)

rowvalue2, colvalue1

(-0.6, -0.2)

In [23]:

def find_dominated_actions(matrix, axis):
    dominated_actions = []
    for i in range(matrix.shape[axis]):
        for j in range(matrix.shape[axis]):
            if i >= j:
                continue
            if np.all(np.take(matrix, i, axis=axis) >= np.take(matrix, j, axis=axis)):
                dominated_actions.append(j)
    return dominated_actions

def find_dominated(matrix1, matrix2):
    dominated_rows = find_dominated_actions(matrix1, axis=0)
    dominated_columns = find_dominated_actions(matrix2, axis=1)
    return dominated_rows, dominated_columns

def iterated_removal_of_dominated_strategies(matrix1, matrix2):
    temp1 = matrix1[:]
    temp2 = matrix2[:]
    while True:
        dominated_rows, dominated_columns = find_dominated(temp1, temp2)
        if len(dominated_rows) + len(dominated_columns) == 0:
            break
    
        non_dominated_mask = np.ones(temp1.shape[0], dtype=bool)
        non_dominated_mask[dominated_rows] = False
        
        temp1 = temp1[non_dominated_mask]
        temp2 = temp2[non_dominated_mask]

        non_dominated_mask = np.ones(temp1.shape[1], dtype=bool)
        non_dominated_mask[dominated_columns] = False

        temp1 = temp1[:,non_dominated_mask]
        temp2 = temp2[:,non_dominated_mask]

    return temp1, temp2


def check_strat_is_row_strat(strat: np.array):
    assert len(strat.shape) == 2 and strat.shape[0] == 1
    
def check_strat_is_col_strat(strat: np.array):
    assert len(strat.shape) == 2 and strat.shape[1] == 1


matrix1 = np.array([[13,1,7], [4,3,6], [-1,2,8]])
matrix2 = np.array([[3,4,3], [1,3,2], [9,8,-1]])

matrixA, matrixB = iterated_removal_of_dominated_strategies(matrix1=matrix1, matrix2=matrix2)
expected_matrixA = np.array([[13, 1], [4, 3]])
expected_matrixB = np.array([[3, 4], [1, 3]])

assert np.array_equal(matrixA, expected_matrixA)
assert np.array_equal(matrixB, expected_matrixB)

matrix1 = np.array([[10,5,3], [0,4,6], [2,3,2]])
matrix2 = np.array([[4,3,2], [1,6,0], [1,5,8]])

matrixA, matrixB = iterated_removal_of_dominated_strategies(matrix1=matrix1, matrix2=matrix2)
expected_matrixA = np.array([[10]])
expected_matrixB = np.array([[4]])

assert np.array_equal(matrixA, expected_matrixA)
assert np.array_equal(matrixB, expected_matrixB)